<div style="text-align: center; font-size: 24px;"> Uso de Análisis de Covarianza (ANCOVA) en investigación científica <br></div>
<div style="text-align: center;font-size: 24px;"> (Use of covariance analysis (ANCOVA) in scientific research) </div>

<div style="text-align: justify;"> <div style="font-weight: bold;">Resumen. </div>Se presentan las bases del ANálisis de COVArianza (ANCOVA). Se manejan los
propósitos y la aplicación de este método estadístico. Se discuten las técnicas para la estimación
de los contrastes, el control y la disminución del grado de error. Se presentan un ANCOVA
simple mediante un ejemplo de datos reales. Se enfatiza el papel de esta técnica estadística en
fijar el efecto de la variable auxiliar en el experimento. </div>

**_Ejemplo._** <div style="text-align: justify;">Un experimento de fertilizantes con el diseño San Cristóbal (12
tratamientos en cuatro bloques completos al azar), realizado por el IMPA en
la zona de abastecimiento del ingenio Motzorongo, en el estado de Veracruz,
cosechado en plantilla durante la zafra 1977 – 1978, produjo los resultados
de la Tabla 2. En esta tabla la Y es el rendimiento de caña en toneladas por
hectárea, y X es el número observado de tallos molederos por parcela
experimental. Se propone examinar el efecto de los nutrientes sobre el rendimiento de caña, eliminando a través de la técnica de covarianza, el
efecto del número de tallos molederos (Martínez-Garza, 1988). </div>

**Referencia**
Badii, M.H., J. Castillo & A. Wong (2008)  Use of covariance analysis (ANCOVA) in scientific research. Consulted: https://eprints.uanl.mx/12489/1/A3.pdf

![Ejemplo de imagen](./PAM_1.png)

In [ ]:
import pingouin as pg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison

In [ ]:
data = {
    'Tratamientos': list(range(1, 13)),
    'I_Y': [107.5, 89.2, 102.2, 88.1, 121.4, 119.4, 110.6, 106.4, 114.7, 116.4, 96.1, 102.5],
    'I_X': [319, 300, 280, 318, 308, 306, 316, 290, 315, 330, 302, 321],
    'II_Y': [103.6, 102.8, 110.0, 105.0, 100.3, 111.1, 113.6, 120.0, 106.9, 129.2, 107.8, 114.4],
    'II_X': [308, 307, 280, 315, 304, 310, 303, 306, 299, 315, 353, 307],
    'III_Y': [84.4, 84.5, 76.9, 104.7, 111.7, 100.8, 114.7, 88.9, 114.4, 106.4, 106.5, 116.4],
    'III_X': [319, 320, 299, 319, 315, 334, 284, 314, 310, 319, 310, 316],
    'IV_Y': [115.6, 108.1, 87.5, 120.3, 126.1, 119.2, 122.2, 130.0, 115.8, 136.9, 122.8, 126.7],
    'IV_X': [275, 302, 268, 311, 290, 296, 295, 299, 297, 317, 294, 302],
}
df = pd.DataFrame(data)

df['SUMA_Y'] = df.filter(like='_Y').sum(axis=1)
df['SUMA_X'] = df.filter(like='_X').sum(axis=1)
df

In [ ]:
# ancova(data=df, dv='SUMA_Y', covar='I_X', between='Tratamientos')

In [ ]:
data = {
    'Tratamientos': list(range(1, 13)) * 4,
    "Parcela" : ["I"] * 12 + ["II"] * 12 + ["III"] * 12 + ["IV"] * 12,
    'Y': [107.5, 89.2, 102.2, 88.1, 121.4, 119.4, 110.6, 106.4, 114.7, 116.4, 96.1, 102.5, 
          103.6, 102.8, 110.0, 105.0, 100.3, 111.1, 113.6, 120.0, 106.9, 129.2, 107.8, 114.4,
          84.4, 84.5, 76.9, 104.7, 111.7, 100.8, 114.7, 88.9, 114.4, 106.4, 106.5, 116.4, 
          115.6, 108.1, 87.5, 120.3, 126.1, 119.2, 122.2, 130.0, 115.8, 136.9, 122.8, 126.7],
    'X': [319, 300, 280, 318, 308, 306, 316, 290, 315, 330, 302, 321,
          308, 307, 280, 315, 304, 310, 303, 306, 299, 315, 353, 307,
          319, 320, 299, 319, 315, 334, 284, 314, 310, 319, 310, 316,
          275, 302, 268, 311, 290, 296, 295, 299, 297, 317, 294, 302],

}
df = pd.DataFrame(data)
df

In [ ]:
#Cuenta el número de individuos que hay en cada grupo
df.boxplot('Y',by='Tratamientos',rot=90)

# Cumplimiento de supuestos

In [ ]:
# Normalidad
#Normalidad prueba de Shapiro-Wilk
#Ho:Normalidad(p>0.05)
#H1: No normalidad (p<0.05)
#Normalidad en las variables
pg.normality(df, dv='Y', group='Tratamientos')

In [ ]:
# Varianza entre grupos      
#Homocedasticidad prueba de Levene (sin normalidad)
#Ho:Homocedasticidad (p>0.05)
#H1: No Homocedasticidad (p<0.05)
pg.homoscedasticity(df, dv='Y', 
                    group='Tratamientos',method='levene')

In [ ]:
#Homocedasticidad prueba de Bartlett (con normalidad)
#Ho:Homocedasticidad (p>0.05)
#H1: No Homocedasticidad (p<0.05)
pg.homoscedasticity(df, dv='Y', 
                    group='Tratamientos',method='bartlett')

In [ ]:
Y_1 = df[df["Parcela"] == "I"]["Y"].to_numpy()
X_1 = df[df["Parcela"] == "I"]["X"].to_numpy().reshape(-1,1)
Y_2 = df[df["Parcela"] == "II"]["Y"].to_numpy()
X_2 = df[df["Parcela"] == "II"]["X"].to_numpy().reshape(-1,1)
Y_3 = df[df["Parcela"] == "III"]["Y"].to_numpy()
X_3 = df[df["Parcela"] == "III"]["X"].to_numpy().reshape(-1,1)
Y_4 = df[df["Parcela"] == "IV"]["Y"].to_numpy()
X_4 = df[df["Parcela"] == "IV"]["X"].to_numpy().reshape(-1,1)

X_train_1,X_test_1,Y_train_1,Y_test_1=train_test_split(X_1,Y_1, test_size=0.9)
X_train_2,X_test_2,Y_train_2,Y_test_2=train_test_split(X_2,Y_2, test_size=0.9)
X_train_3,X_test_3,Y_train_3,Y_test_3=train_test_split(X_3,Y_3, test_size=0.9)
X_train_4,X_test_4,Y_train_4,Y_test_4=train_test_split(X_4,Y_4, test_size=0.9)

rls=linear_model.LinearRegression()
modelo_1=rls.fit(X_train_1,Y_train_1)
Y_pred_1=rls.predict(X_test_1)
modelo_2=rls.fit(X_train_2,Y_train_2)
Y_pred_2=rls.predict(X_test_2)
modelo_3=rls.fit(X_train_3,Y_train_3)
Y_pred_3=rls.predict(X_test_3)
modelo_4=rls.fit(X_train_4,Y_train_4)
Y_pred_4=rls.predict(X_test_4)

#Gráfica del modelo
plt.scatter(X_test_1[:, 0], Y_test_1, label='Valores reales 1')
plt.plot(X_test_1[:, 0], Y_pred_1, color='r', label='RL 1', linewidth=3)
plt.scatter(X_test_2[:, 0], Y_test_2, label='Valores reales 2')
plt.plot(X_test_2[:, 0], Y_pred_2, color='r', label='RL 2', linewidth=3)
plt.scatter(X_test_3[:, 0], Y_test_3, label='Valores reales 3')
plt.plot(X_test_3[:, 0], Y_pred_3, color='r', label='RL 3', linewidth=3)
plt.scatter(X_test_4[:, 0], Y_test_4, label='Valores reales 4')
plt.plot(X_test_4[:, 0], Y_pred_4, color='r', label='RL 4', linewidth=3)
plt.title('Regresión Lineal')
plt.xlabel('Tallos molederos')
plt.ylabel('Hectáreas Totales')
plt.legend()
plt.show()

In [ ]:
model = ols('Y ~ C(Tratamientos) * X', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# ANCOVA

In [ ]:
model = ols('Y ~ C(Tratamientos) + X', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# PINGOUIN.ANCOVA

#### Parameters 
**datapandas.DataFrame**<br>
DataFrame. Note that this function can also directly be used as a Pandas method, in which case this argument is no longer needed.<br>

**dvstring**<br>
Name of column in data with the dependent variable.<br>

**betweenstring**<br>
Name of column in data with the between factor.<br>

**covarstring or list**<br>
Name(s) of column(s) in data with the covariate.<br>

**effsizestr**<br>
Effect size. Must be ‘np2’ (partial eta-squared) or ‘n2’ (eta-squared).<br>

#### Returns
aovpandas.DataFrame<br>
ANCOVA summary:

'Source': Names of the factor considered

'SS': Sums of squares

'DF': Degrees of freedom

'F': F-values

'p-unc': Uncorrected p-values

'np2': Partial eta-squared

In [ ]:
pg.ancova(data=df, dv='Y', covar='X', between='Tratamientos')

# POST HOC

In [ ]:
# Realizar una prueba de Tukey
comp = MultiComparison(df['Y'], df['Tratamientos'])
post_hoc_res = comp.tukeyhsd()
print(post_hoc_res.summary())

In [ ]:
pg.pairwise_tukey(data=df, dv='Y', between='Tratamientos')

# PRÁCTICA

In [ ]:
# from pingouin import ancova, read_dataset
# df = read_dataset('ancova')
# df

In [ ]:
# ancova(data=df, dv='Scores', covar='Income', between='Method')

In [ ]:
# ancova(data=df, dv='Scores', covar=['Income', 'BMI'], between='Method',
#        effsize="n2")